In [1]:
import socket
import numpy as np
import time
import psycopg2
from influxdb_client import InfluxDBClient
import warnings
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc_file_defaults()
def figure(name, *args, **kwargs):
    plt.close(name)
    return plt.figure(name, *args, **kwargs)
from matplotlib.colors import ListedColormap
colorblindmap = ListedColormap(['#006BA4', '#FF800E', '#ABABAB', '#595959', '#5F9ED1', '#C85200', '#898989', '#A2C8EC', '#FFBC79', '#CFCFCF'])

'''
conn = psycopg2.connect(database="fsexp",
                        host="172.30.2.184",
                        user="postgres",
                        password="6Fd9d8P7KixkPA",
                        port="5432")
cursor = conn.cursor()
tokenRead = "4voZq4pLkjT4rpAvsJMZgtc6xNrgWhXplnxfk9BycfBC-wpBDyvAxaBisrzaSBdB1ILXsq19l4bptacrgPzpmQ=="
clienRead = InfluxDBClient(url="http://spikedb.alpvision.com:8086", token=tokenRead, timeout=60_000)
query_api = clienRead.query_api()'''

'\nconn = psycopg2.connect(database="fsexp",\n                        host="172.30.2.184",\n                        user="postgres",\n                        password="6Fd9d8P7KixkPA",\n                        port="5432")\ncursor = conn.cursor()\ntokenRead = "4voZq4pLkjT4rpAvsJMZgtc6xNrgWhXplnxfk9BycfBC-wpBDyvAxaBisrzaSBdB1ILXsq19l4bptacrgPzpmQ=="\nclienRead = InfluxDBClient(url="http://spikedb.alpvision.com:8086", token=tokenRead, timeout=60_000)\nquery_api = clienRead.query_api()'

In [3]:
import os
import glob
import sys
sys.path.append('/home/vincent/Downloads/load_intan_rhs_format/')
from load_intan_rhs_format import read_data

def raw_data_100ms(filename: str, offset_ms: int):
    nb_sample = 3000 # 100ms
    offset = 30*offset_ms
    rawdatas = read_data(filename)
    digin = rawdatas['spike_triggers']
    triggers_raw = np.argwhere(digin)
    raw_amp = rawdatas['amplifier_data']
    triggers = [triggers_raw[0, :]]
    for i in range(triggers_raw.shape[0]-1):
        t1 = triggers_raw[i,0]
        t2 = triggers_raw[i+1,0]
        if t2-t1 > 100:
            triggers.append(triggers_raw[i+1,:])
    
    raws100ms_trigger = {}
    for i in range(8):
        raws100ms_trigger[i] = []
    for i in range(len(triggers)):
        ind = triggers[i][0]
        if ind+nb_sample<raw_amp.shape[1]:
            raws100ms_trigger[int(triggers[i][0])].append(raw_amp[:,ind+offset:ind + nb_sample + offset])
    for i in range(8):
        raws100ms_trigger[i] = np.array(raws100ms_trigger[i])
    return raws100ms_trigger

def save_raw_data(raws: dict, filename: str):
    np.savez_compressed(filename, t0=raws[0],t1=raws[1],t2=raws[2],t3=raws[3],t4=raws[4],t5=raws[5],t6=raws[6],t7=raws[7])

def load_raw_data(filename: str):
    list_array = []
    for i in range(8):
        list_array.append(f't{i}')
    
    loaded = np.load(filename)
    raws = {}
    for i,aname in enumerate(list_array):
        raws[i] = loaded[aname]
    
    return raws

In [4]:
dirPath = 'experiment_data/'
listFiles = filter(os.path.isfile,glob.glob(f'{dirPath}/*.rhs'))
listFiles = sorted(listFiles, key=os.path.getmtime)
print(len(listFiles))

2


In [5]:
offset_trigger_ms = 5
#for i,filename in enumerate(tqdm(listFiles)):
filename = dirPath + 'exp_240213_151732.rhs'
r = raw_data_100ms(filename, offset_trigger_ms)
#savename = dirPath + f'/exp_{i//2}_{i%2}_{offset_trigger_ms}.npz'
savename = dirPath + f'/exp_0_0_{offset_trigger_ms}.npz'
save_raw_data(r, savename)


Reading Intan Technologies RHS Data File, Version 1.0

Found 32 amplifier channels.
Found 0 board ADC channels.
Found 8 board DAC channels.
Found 0 board digital input channels.
Found 0 board digital output channels.



FileSizeError: Something is wrong with file size : should have a whole number of data blocks

In [ ]:
matplotlib.rc_file_defaults()


def float_to_rgb(val: float, start_rgb: np.array, end_rgb: np.array):
    r_diff = end_rgb[0] - start_rgb[0]
    g_diff = end_rgb[1] - start_rgb[1]
    b_diff = end_rgb[2] - start_rgb[2]
    return [start_rgb[0] + val * r_diff, start_rgb[1] + val * g_diff, start_rgb[2] + val * b_diff]
    
def subplot_raw_map(ax: plt.Axes, raw1: np.array, raw2: np.array, title: str):
    if raw1.shape[1] != raw2.shape[1]:
        print(f'Error shape raw1/raw2')
    sampling = 30
    nb_points = raw1.shape[1] // sampling
    raw_map = np.ones((raw1.shape[0] + 1 + raw2.shape[0], nb_points, 3))
    MAXAMP = 100
    r1 = raw1/MAXAMP
    r1[r1 > 1.0] = 1.0
    r1[r1 < -1.0] = -1.0
    r2 = raw2/MAXAMP
    r2[r2 > 1.0] = 1.0
    r2[r2 < -1.0] = -1.0
    
    for i in range(raw1.shape[0]):
        for j in range(nb_points):
            rs = r1[i,j*sampling:j*sampling+sampling]
            rmax = np.abs(np.amax(rs))
            rmin = np.abs(np.amin(rs))
            if rmax > rmin:
                raw_map[i, j, :] = float_to_rgb(rmax, [1.0, 1.0, 1.0], [1.0, 0.0, 0.0])
            else:
                raw_map[i, j, :] = float_to_rgb(rmin, [1.0, 1.0, 1.0], [0.0, 0.0, 1.0])

    # line
    raw_map[raw1.shape[0], :, :] = 0.0
    
    for i in range(raw2.shape[0]):
        for j in range(nb_points):
            rs = r2[i,j*sampling:j*sampling+sampling]
            rmax = np.abs(np.amax(rs))
            rmin = np.abs(np.amin(rs))
            if rmax > rmin:
                raw_map[i + raw1.shape[0] + 1, j, :] = float_to_rgb(rmax, [1.0, 1.0, 1.0], [1.0, 0.0, 0.0])
            else:
                raw_map[i + raw1.shape[0] + 1, j, :] = float_to_rgb(rmin, [1.0, 1.0, 1.0], [0.0, 0.0, 1.0])
                
    ax.imshow(raw_map)
    ax.set_title(title)

def plot_ns_exp(nsid: int, start_exp_index: int, offset: int):
    title = f'NS {nsid} - Exp {start_exp_index}'
    nb_col_plt = 4
    nb_row_plt = 3 * 8
    fig, axs = plt.subplots(nb_row_plt, nb_col_plt, num=title, figsize=(20, 2 * nb_row_plt), constrained_layout=True)

    map8 = np.array([[1,0],[0,1],[1,1],[1,2],[2,2],[2,1],[3,1],[2,0]])
    elecs = [nsid*8+i for i in range(8)]

    fig.suptitle(title)
    raw1 = load_raw_data(f'{dirPath}/exp_{start_exp_index}_0_{offset}.npz')
    raw2 = load_raw_data(f'{dirPath}/exp_{start_exp_index}_1_{offset}.npz')

    for j in range(8):
        for i,elec in enumerate(elecs):
            if i==j:
                title_elec = f'Electrode {elec}*'
            else:
                title_elec = f'Electrode {elec}'
            subplot_raw_map(axs[map8[i,1] + j * 3,map8[i,0]], raw1[j][:, elec-32, :], raw2[j][:, elec-32, :], title_elec)

        axs[j*3,0].axis('off')
        axs[j*3,-1].axis('off')
        axs[j*3+2,0].axis('off')
        axs[j*3+2,-1].axis('off')

    for ax in axs.flat:
        ax.set(xlabel='ms', ylabel='Exp')
    for ax in axs.flat:
        ax.label_outer()